evaluation and results print

In [ ]:

def evaluate(self):
    print(json.dumps(vars(self.params), indent=2))

    model: torch.nn.Module = Classifier(self.params, vocab_size=len(self.data_loader.w2i), pte=None)
    if self.utils.HALF:
        model.half()
    if torch.cuda.is_available():
        model = model.cuda()
    # Load the model weights
    model.load_state_dict(torch.load("ckpt/" + self.params.model_file, map_location=lambda storage, loc: storage))

    model.eval()

    # This dataset is only available for the binary classifier
    if self.params.ntags == 2:
        accuracy, all_actual, all_predicted = self._evaluate_aux(model, self.data_loader.test_data_loader)
        prec_mac, recall_mac, f1_mac, _ = precision_recall_fscore_support(all_actual, all_predicted, average='macro')
        prec_mic, recall_mic, f1_mic, _ = precision_recall_fscore_support(all_actual, all_predicted, average='micro')
        print("Accuracy on the OOD test set 1: {:.4f}".format(accuracy))
        print("Precision on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(prec_mac, prec_mic))
        print("Recall on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(recall_mac, recall_mic))
        print("F1 on the OOD test set 1 macro / micro: {:.4f}, {:.4f}".format(f1_mac, f1_mic))
        print("Latex: {:5.2f} & {:5.2f} & {:5.2f} & {:5.2f}".format(accuracy*100, prec_mac*100, recall_mac*100, f1_mac*100))
        print("----------------------------------------------------------------------")

    accuracy, all_actual, all_predicted = self._evaluate_aux(model, self.data_loader.test_data_loader_2)
    prec_mac, recall_mac, f1_mac, _ = precision_recall_fscore_support(all_actual, all_predicted, average='macro')
    prec_mic, recall_mic, f1_mic, _ = precision_recall_fscore_support(all_actual, all_predicted, average='micro')
    print("Accuracy on the OOD test set 2: {:.4f}".format(accuracy))
    print("Precision on the OOD test set 2 macro / micro: {:.4f}, {:.4f}".format(prec_mac, prec_mic))
    print("Recall on the OOD test set 2 macro / micro: {:.4f}, {:.4f}".format(recall_mac, recall_mic))
    print("F1 on the OOD test set 2 macro / micro: {:.4f}, {:.4f}".format(f1_mac, f1_mic))
    print("Latex: {:5.2f} & {:5.2f} & {:5.2f} & {:5.2f}".format(accuracy * 100, prec_mac * 100, recall_mac * 100, f1_mac * 100))

    if self.params.ntags == 4:
        results = confusion_matrix(all_actual, all_predicted)
        df_cm = pd.DataFrame(results, index=[i for i in ["Satire", "Hoax", "Propaganda", "Trusted"]],
                             columns=[i for i in ["Satire", "Hoax", "Propaganda", "Trusted"]])
        sns_plot = sn.heatmap(df_cm, annot=True, fmt='g')
        plt.yticks(rotation=45)
        sns_plot.get_figure().savefig('plots/cm.png')

    print("----------------------------------------------------------------------")
    accuracy, all_actual, all_predicted = self._evaluate_aux(model, self.data_loader.dev_data_loader)
    prec_mac, recall_mac, f1_mac, _ = precision_recall_fscore_support(all_actual, all_predicted, average='macro')
    prec_mic, recall_mic, f1_mic, _ = precision_recall_fscore_support(all_actual, all_predicted, average='micro')
    print("Accuracy on the dev set: {:.4f}".format(accuracy))
    print("Precision on the dev set macro / micro: {:.4f}, {:.4f}".format(prec_mac, prec_mic))
    print("Recall on the dev macro / micro: {:.4f}, {:.4f}".format(recall_mac, recall_mic))
    print("F1 on the dev macro / micro: {:.4f}, {:.4f}".format(f1_mac, f1_mic))
    print("Latex: {:5.2f} & {:5.2f} & {:5.2f} & {:5.2f}".format(accuracy * 100, prec_mac * 100, recall_mac * 100, f1_mac * 100))


for 4-way classification, results:

Total time taken (in seconds): 5124.499713159166 (1.4 hours)

model configuration

In [ ]:
{
  "root": "./data/fakeNews/",
  "train": "./data/fakeNews/fulltrain.csv",
  "dev": "./data/fakeNews/balancedtest.csv",
  "test": "./data/fakeNews/test.xlsx",
  "pte": "",
  "entity_desc": "./data/fakeNews/entityDescCorpus.pkl",
  "entity_tran": "./data/fakeNews/entity_feature_transE.pkl",
  "adjs": "./data/fakeNews/adjs/",
  "emb_dim": 100,
  "hidden_dim": 100,
  "node_emb_dim": 32,
  "max_epochs": 15,
  "max_sent_len": 50,
  "max_sents_in_a_doc": 10000,
  "batch_size": 32,
  "lr": 0.001,
  "dropout": 0.5,
  "ntags": 4,
  "weight_decay": 1e-06,
  "pooling": "max",
  "model_file": "model_CompareNet_Max_DET_0322_1528.t7",
  "plot": 0,
  "mode": 0,
  "cuda": true,
  "device": 0,
  "HALF": true,
  "DEBUG": false,
  "node_type": 3,
  "repeat": 1,
  "seed": [
    5
  ],
  "config": "CompareNet_Max_DET_0322_1528"
}

results print:

OOD test set 2 means LUN-test dataset, corresponding to "params.dev" in parameter setting, corresponding to balancedtext.csv file
the dev means 20% sampling from the LUN-training dataset, corresponding to "fulltrain.csv" file.

In [ ]:
Accuracy on the OOD test set 2: 0.6130
Precision on the OOD test set 2 macro / micro: 0.6496, 0.6130
Recall on the OOD test set 2 macro / micro: 0.6130, 0.6130
F1 on the OOD test set 2 macro / micro: 0.6007, 0.6130
Latex: 61.30 & 64.96 & 61.30 & 60.07
----------------------------------------------------------------------
Accuracy on the dev set: 0.9566
Precision on the dev set macro / micro: 0.9558, 0.9566
Recall on the dev macro / micro: 0.9564, 0.9566
F1 on the dev macro / micro: 0.9560, 0.9566
Latex: 95.66 & 95.58 & 95.64 & 95.60

for 2-way classification, results:

Total time taken (in seconds): 6714.280589981936 (1.9 hours)

model configuration

In [ ]:
{
  "root": "./data/fakeNews/",
  "train": "./data/fakeNews/fulltrain.csv",
  "dev": "./data/fakeNews/balancedtest.csv",
  "test": "./data/fakeNews/test.xlsx",
  "pte": "",
  "entity_desc": "./data/fakeNews/entityDescCorpus.pkl",
  "entity_tran": "./data/fakeNews/entity_feature_transE.pkl",
  "adjs": "./data/fakeNews/adjs/",
  "emb_dim": 100,
  "hidden_dim": 100,
  "node_emb_dim": 32,
  "max_epochs": 15,
  "max_sent_len": 50,
  "max_sents_in_a_doc": 10000,
  "batch_size": 32,
  "lr": 0.001,
  "dropout": 0.5,
  "ntags": 2,
  "weight_decay": 1e-06,
  "pooling": "max",
  "model_file": "model_CompareNet_Max_DET_0302_0354.t7",
  "plot": 0,
  "mode": 0,
  "cuda": true,
  "device": 0,
  "HALF": true,
  "DEBUG": false,
  "node_type": 3,
  "repeat": 1,
  "seed": [
    5
  ],
  "config": "CompareNet_Max_DET_0302_0354"
}

results print:

OOD test set 1 means SLN testing dataset, corresponding to "params.test" in parameter setting, corresponding to text.xlsx file

In [ ]:
Accuracy on the OOD test set 1: 0.8667
Precision on the OOD test set 1 macro / micro: 0.8668, 0.8667
Recall on the OOD test set 1 macro / micro: 0.8667, 0.8667
F1 on the OOD test set 1 macro / micro: 0.8667, 0.8667
Latex: 86.67 & 86.68 & 86.67 & 86.67
----------------------------------------------------------------------
Accuracy on the OOD test set 2: 0.9185
Precision on the OOD test set 2 macro / micro: 0.9185, 0.9185
Recall on the OOD test set 2 macro / micro: 0.9186, 0.9185
F1 on the OOD test set 2 macro / micro: 0.9185, 0.9185
Latex: 91.85 & 91.85 & 91.86 & 91.85
----------------------------------------------------------------------
Accuracy on the dev set: 0.9668
Precision on the dev set macro / micro: 0.9684, 0.9668
Recall on the dev macro / micro: 0.9630, 0.9668
F1 on the dev macro / micro: 0.9655, 0.9668
Latex: 96.68 & 96.84 & 96.30 & 96.55